In [1]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import time
import threading

client = Client("IRIS")
starttime = UTCDateTime.utcnow() - 60
endtime = UTCDateTime.utcnow() - 59  


stream_buffer = None
buffer_lock = threading.Lock()

def fetch_data():
    global starttime, endtime, stream_buffer
    for _ in range(100):
        try:
            st = client.get_waveforms("IN", "SHL", "*", "HH?", starttime, endtime)
            st.merge()
            with buffer_lock:
                stream_buffer = st.copy()  
            starttime = endtime
            endtime += 1
            time.sleep(1)
        except Exception as e:
            print(f"Fetch error: {e}")
            time.sleep(5)

def process_data():
    while True:
        with buffer_lock:
            if stream_buffer is not None:
                st_proc = stream_buffer.copy()
                st_proc.filter("bandpass", freqmin=1.0, freqmax=10.0)
                
        time.sleep(0.5)  

threading.Thread(target=fetch_data, daemon=True).start()
threading.Thread(target=process_data, daemon=True).start()

while True:
    with buffer_lock:
        if stream_buffer:
            stream_buffer.plot(size=(800, 600), handle=True)
    time.sleep(1)   

ModuleNotFoundError: No module named 'pkg_resources'

In [21]:
from obspy.clients.seedlink import Client
client = Client("geofon.gfz.de") 
stations=client.get_info() 

In [1]:
from obspy.clients.seedlink.easyseedlink import create_client
from xml.etree import ElementTree as ET

def handle_data(trace):
    print(trace)
    trace.plot()

client = create_client("geofon.gfz-potsdam.de:18000", on_data=handle_data)


stations_xml = client.get_info('STATIONS')
root = ET.fromstring(stations_xml)

stations = [(s.get("network"), s.get("name")) for s in root.findall(".//station")]



d:\Project_E\backend\.venv\lib\site-packages\obspy\core\util\base.py:26: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


SeedLinkException: 'cannot connect to SeedLink server: [Errno 11001] getaddrinfo failed'

In [2]:
net,sta=stations[0]

client.select_stream(net, sta, "BHE")
client.select_stream(net, sta, "BHN")
client.select_stream(net, sta, "BHZ")

client.run()   

NameError: name 'stations' is not defined